In [1]:
import gymnasium as gym
import torch
from torch.utils.tensorboard import SummaryWriter
import tianshou as ts

In [2]:
task = 'CartPole-v1'
lr, epoch, batch_size = 1e-3, 10, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, 10
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))  # TensorBoard is supported!
# For other loggers: https://tianshou.readthedocs.io/en/master/tutorials/logger.html

In [3]:
# you can also try with SubprocVectorEnv
train_envs = ts.env.DummyVectorEnv([lambda: gym.make(task) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make(task) for _ in range(test_num)])

In [4]:
from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/master/tutorials/dqn.html#build-the-network
env = gym.make(task, render_mode="human")
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[128, 128, 128])
optim = torch.optim.Adam(net.parameters(), lr=lr)

In [5]:
policy = ts.policy.DQNPolicy(
    model=net,
    optim=optim,
    discount_factor=gamma, 
    action_space=env.action_space,
    estimation_step=n_step,
    target_update_freq=target_freq
)
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(buffer_size, train_num), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)  # because DQN uses epsilon-greedy method

In [6]:
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=epoch,
    step_per_epoch=step_per_epoch,
    step_per_collect=step_per_collect,
    episode_per_test=test_num,
    batch_size=batch_size,
    update_per_step=1 / step_per_collect,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    logger=logger,
).run()
print(f"Finished training in {result.timing.total_time} seconds")

Epoch #1: 10001it [00:05, 1722.65it/s, env_step=10000, gradient_step=1000, len=174, n/ep=0, n/st=10, rew=174.00]                           


Epoch #1: test_reward: 224.180000 ± 78.557289, best_reward: 224.180000 ± 78.557289 in #1


Epoch #2: 10001it [00:05, 1734.55it/s, env_step=20000, gradient_step=2000, len=381, n/ep=0, n/st=10, rew=381.00]                           


Epoch #2: test_reward: 279.760000 ± 80.052373, best_reward: 279.760000 ± 80.052373 in #2


Epoch #3: 10001it [00:05, 1785.74it/s, env_step=30000, gradient_step=3000, len=209, n/ep=0, n/st=10, rew=209.00]                           


...

Epoch #6: test_reward: 274.790000 ± 64.178235, best_reward: 279.760000 ± 80.052373 in #2


Epoch #7: 10001it [00:05, 1739.92it/s, env_step=70000, gradient_step=7000, len=207, n/ep=0, n/st=10, rew=207.00]                           


Epoch #7: test_reward: 209.400000 ± 18.074844, best_reward: 279.760000 ± 80.052373 in #2


Epoch #8: 10001it [00:05, 1729.35it/s, env_step=80000, gradient_step=8000, len=291, n/ep=0, n/st=10, rew=291.00]                           


Epoch #8: test_reward: 493.340000 ± 38.675372, best_reward: 493.340000 ± 38.675372 in #8
Finished training in 57.980472564697266 seconds


In [7]:
torch.save(policy.state_dict(), 'dqn.pth')
policy.load_state_dict(torch.load('dqn.pth'))

<All keys matched successfully>

In [8]:
policy.eval()
policy.set_eps(eps_test)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

CollectStats(n_collected_episodes=1, n_collected_steps=500, collect_time=25.654741287231445, collect_speed=19.489574827591564, returns=array([500.]), returns_stat=SequenceSummaryStats(mean=500.0, std=0.0, max=500.0, min=500.0), lens=array([500]), lens_stat=SequenceSummaryStats(mean=500.0, std=0.0, max=500.0, min=500.0))